In [76]:
import numpy as np
from scipy import optimize
import os
import subprocess
from contextlib import contextmanager
import time


First, we'll read in the files created by the Fortran HF code. Things are a little messy thanks to the way I stored things in the past, but they are standard numpy arrays after this cell. Take note of the ordering and suggest a more agreeable choice if you've got an opinion.

In [102]:
@contextmanager
def cwd(path):
    oldpwd=os.getcwd()
    os.chdir(path)
    try:
        yield
    finally:
        os.chdir(oldpwd)

def spin(spin):
    if(spin==1): return -0.5
    return 0.5

def make_input(params,restart):
    in_template = "../HF/in.template"
    f1 = open(in_template, 'r')
    
    int_string = "&interaction t0={t0},x0={x0},t1={t1},x1={x1},t2={t2},x2={x2},t3={t3},x3={x3},w0={w0},sig={sig},icoul=1,icm=1,j2terms=.true. / \n".format(\
    t0=params["t0"],x0=params["x0"],t1=params["t1"],x1=params["x1"],t2=params["t2"],x2=params["x2"],t3=params["t3"],x3=params["x3"],w0=params["w0"],sig=params["sig"])
    if(restart):
        io_string = "&io printwf=F,printdens=F,restartwf=T,printhpsi=T /"
    else:
        io_string = "&io printwf=T,printdens=F,restartwf=F,printhpsi=F /"
    in_out = "../HF/in"
    f2 = open(in_out, 'w')
    f1.seek(0)
    f2.write(f1.read())
    f2.write(int_string)
    f2.write(io_string)
    f2.write("\n")

    f1.close()
    f2.close()

    return

def run_HF(tag,restart):
    noNaN = 1
    if(restart == 0):
        os.system("cp in results/in."+tag)
        #print(os.system("../HF/./main"))
        output = subprocess.run(["./main"], capture_output=True)
        #print(output)
        #print(subprocess.run("../HF/./main",shell=True))
        os.system("mv out results/out."+tag)
        os.system("mv wf_numpy.bin results/wf_numpy.bin."+tag)
        noNaN = os.system("grep -q NaN results/out."+tag)
    else:
        output = subprocess.run(["./main"], capture_output=True)
        #os.system("mv out results/out_rst."+tag)
        #os.system("mv hpsi.bin results/hpsi.bin."+tag)
        #print(output)
        #noNaN = os.system("grep -q NaN results/out_rst."+tag)
    if (noNaN == 0):
        print("NaNs in run {tag}!".format(tag=tag))
        return 1

    return 0


def read_wfs(restartfile):
    #restartfile = '../HF/wf_numpy.bin'

    intchunk = np.dtype(np.int32).itemsize
    realchunk = np.dtype(np.float64).itemsize
    h = np.fromfile(restartfile, dtype=np.float64, count=1, offset=0)[0]
    nbox = np.fromfile(restartfile, dtype=np.int32, count=1, offset=(realchunk+0*intchunk))[0]
    lmax = np.fromfile(restartfile, dtype=np.int32, count=1, offset=(realchunk+1*intchunk))[0]
    nmax = np.fromfile(restartfile, dtype=np.int32, count=1, offset=(realchunk+2*intchunk))[0]
#     print("hbox = ", h)
#     print("nbox = ",nbox)
#     print("lmax = ",lmax)
#     print("nmax = ",nmax)

    #sortstates, sortenergies, wfr,
    sortstates = np.fromfile(restartfile, dtype=np.int32, count=nmax*3*2, offset=(realchunk+3*intchunk)).reshape(2,3,nmax)
    sortenergies = np.fromfile(restartfile, dtype=np.float64, count=nmax*2, offset=(realchunk+3*intchunk+nmax*3*2*intchunk)).reshape(2,nmax)
    wfs = np.fromfile(restartfile, dtype=np.float64, count=2*2*(lmax+1)*lmax*(nbox+1), offset=(realchunk+3*intchunk+nmax*3*2*intchunk+sortenergies.nbytes)).reshape(2,2,(lmax+1),lmax,(nbox+1))

    return h,nbox,lmax,nmax,sortstates,sortenergies,wfs

def print_wfs(restartfile,h,nbox,lmax,nmax,sortstates,sortenergies,wfs):
    with open(restartfile, 'wb') as file:
        file.write(h)
        file.write(nbox)
        file.write(lmax)
        file.write(nmax)
        file.write(sortstates.reshape((nmax*3*2,)))
        file.write(sortenergies.reshape((nmax*2,)))
        file.write(wfs.reshape((2*2*(lmax+1)*lmax*(nbox+1),)))

    return

def read_hpsi(restartfile):

    intchunk = np.dtype(np.int32).itemsize

    nbox = np.fromfile(restartfile, dtype=np.int32, count=1, offset=(0*intchunk))[0]
    lmax = np.fromfile(restartfile, dtype=np.int32, count=1, offset=(1*intchunk))[0]
    nmax = np.fromfile(restartfile, dtype=np.int32, count=1, offset=(2*intchunk))[0]

    sortenergies = np.fromfile(restartfile, dtype=np.float64, count=nmax*2, offset=(3*intchunk)).reshape(2,nmax)
    hpsi = np.fromfile(restartfile, dtype=np.float64, count=2*2*(lmax+1)*lmax*(nbox+1), offset=(3*intchunk+sortenergies.nbytes)).reshape(2,2,(lmax+1),lmax,(nbox+1))

    return sortenergies,hpsi

def get_hpsi(params,h,nbox,lmax,nmax,sortstates,sortenergies,wf):
    restartfile = '../HF/wf_numpy.bin'
    print_wfs(restartfile,h,nbox,lmax,nmax,sortstates,sortenergies,wf)
    make_input(params,1)
    #with cwd('../HF/'):
    oldpwd=os.getcwd()
    os.chdir('../HF/')
    errors.append(run_HF("None",1))
    os.chdir(oldpwd)
    restartfile = '../HF/hpsi.bin'
    sortenergies,hpsi = read_hpsi(restartfile)

    return hpsi

def make_wf(coeffs,train_wfs,states,levels):
    train_num = len(train_wfs)
    tot_levels = sum(levels)
    trial_wf = np.zeros(train_wfs[0].shape)
    for i in range(train_num):
        for isospin in range(2):
            for j in range(levels[isospin]):
                i_s = sortstates[isospin,2,j]-1
                i_l = sortstates[isospin,1,j]
                i_n = sortstates[isospin,0,j]-1
                trial_wf[isospin,i_s,i_l,i_n,:] = trial_wf[isospin,i_s,i_l,i_n,:] \
                    + train_wfs[i][isospin,i_s,i_l,i_n,:] * coeffs[i*tot_levels + isospin*levels[0] + j]
    return trial_wf

def galerkin(coeffs,train_wfs,states,levels,params):
    
    global count
    global prev_time
    
    count += 1
    print("Iteration {}".format(count))
    print("Iteration time: {}".format(time.time() - prev_time))
    
    prev_time = time.time()
    
    results = np.zeros(coeffs.shape)
    
    train_num = len(train_wfs)
    tot_levels = sum(levels)
    
    trial_wf = make_wf(coeffs,train_wfs,states,levels)

    hpsi = get_hpsi(params,h,nbox,lmax,nmax,sortstates,sortenergies,trial_wf)

    for i in range(train_num):
        for isospin in range(2):
            for j in range(levels[isospin]):
                i_s = sortstates[isospin,2,j]-1
                i_l = sortstates[isospin,1,j]
                i_n = sortstates[isospin,0,j]-1
                ele = i*tot_levels + isospin*levels[0] + j
                lam = coeffs[train_num*tot_levels + isospin*levels[0] + j]
                results[ele] = h*np.dot(train_wfs[i][isospin,i_s,i_l,i_n,:],hpsi[isospin,i_s,i_l,i_n,:]) \
                    - lam*h*np.dot(train_wfs[i][isospin,i_s,i_l,i_n,:],trial_wf[isospin,i_s,i_l,i_n,:])


    for isospin in range(2):
        for j in range(levels[isospin]):
            i_s = sortstates[isospin,2,j]-1
            i_l = sortstates[isospin,1,j]
            i_n = sortstates[isospin,0,j]-1
            ele = train_num*tot_levels + isospin*levels[0] + j
            results[ele] = h*np.dot(trial_wf[isospin,i_s,i_l,i_n,:],trial_wf[isospin,i_s,i_l,i_n,:]) - 1.0
            
    return results


The following cell defines a list of parameters (only one right now) and then loops over those parameters and runs the HF code to generate outputs and WFs.

The inputs, outputs, and WFs are copied to a results folder (this can be changed) with some tag appended to the file name. In the future we may want to change how this tag is generated.

In [120]:
# params = [[-2483.45,13757.0],\
#           [-2503.45,13757.0],\
#           [-2353.45,14757.0],\
#           [-2583.45,12757.0],\
#           [-2383.45,13457.0],]

default_params = {"t0":-2483.45,"x0":0.77600,"t1":484.2300,"x1":-0.31700,"t2":-556.6900,"x2":-1.0,"t3":13757.0,"x3":1.2630,"w0":125.0,"sig":6.0}
params = [\
          {"t0":-2263.45,"x0":0.77600,"t1":484.2300,"x1":-0.31700,"t2":-556.6900,"x2":-1.0,"t3":12757.0,"x3":1.2630,"w0":125.0,"sig":6.0},\
          {"t0":-2383.45,"x0":0.77600,"t1":484.2300,"x1":-0.31700,"t2":-556.6900,"x2":-1.0,"t3":13757.0,"x3":1.2630,"w0":125.0,"sig":6.0},\
          {"t0":-2683.45,"x0":0.77600,"t1":484.2300,"x1":-0.31700,"t2":-556.6900,"x2":-1.0,"t3":12757.0,"x3":1.2630,"w0":125.0,"sig":6.0},\
          {"t0":-2683.45,"x0":0.77600,"t1":484.2300,"x1":-0.31700,"t2":-556.6900,"x2":-1.0,"t3":14757.0,"x3":1.2630,"w0":125.0,"sig":6.0},\
          {"t0":-2483.45,"x0":0.77600,"t1":484.2300,"x1":-0.31700,"t2":-556.6900,"x2":-1.0,"t3":13757.0,"x3":1.2630,"w0":125.0,"sig":6.0},\
         ]
errors = []

for i in range(len(params)):
    make_input(params[i],0)
    with cwd('../HF/'):
        errors.append(run_HF(str(i),0))




Now we read the files back in and print the energies of the single particle states to see whats up.

In [121]:
#params = [[-2483.45],[-2485.45],[-2453.45],[-2583.45],[-1983.45],]

train_wfs = []
states = []
energies = []
levels = [0,0]
for i in range(len(params)):
    if(errors[i] == 1): continue
    restartfile = '../HF/results/wf_numpy.bin.'+str(i)
    h,nbox,lmax,nmax,sortstates,sortenergies,wf = read_wfs(restartfile)
    train_wfs.append(wf)
    states.append(sortstates)
    hpsi = get_hpsi(params[i],h,nbox,lmax,nmax,sortstates,sortenergies,wf)

    #Calculate energies as a test for levels
    energy = [[],[]]
    for isospin in range(2):
        occupied = 0
        for i in range(lmax):
            level = i
            i_s = sortstates[isospin,2,level]-1
            i_l = sortstates[isospin,1,level]
            i_n = sortstates[isospin,0,level]-1
            if(i_s < 0): continue
            occupied += 1
            sp_en = np.sum(-h*wf[isospin,i_s,i_l,i_n,:]*hpsi[isospin,i_s,i_l,i_n,:])
            energy[isospin].append(sp_en)
            #print("Level: {}, N: {}, L: {}, S: {}".format(level+1,i_n+1,i_l,i_s))
            #print("Energy: {}".format(energy[isospin][i]))
        levels[isospin] = occupied
    energies.append(energy)

#h,nbox,lmax,nmax,sortstates,sortenergies,wf = read_wfs()
# npart = 0
# for i in range(nmax):
#     if(sortstates[0,0,i]>0):
#         nfill = int(2*(sortstates[0,1,i]+spin(sortstates[0,2,i])) + 1)
#         if (sortstates[0,1,i] == 0): nfill = 2
#         npart += nfill
#         print(npart)

Now we can calculate the hpsi (or, G) objects from Wfs. Here I am reading in training Wfs just as a test.

In [122]:
for i in range(len(params)):
    if(errors[i] == 1): continue
    # The following is to simply read in the trial WFs during testing
    restartfile = '../HF/results/wf_numpy.bin.'+str(i)
    h,nbox,lmax,nmax,sortstates,sortenergies,wf = read_wfs(restartfile)

    # get hpsi
    hpsi = get_hpsi(params[i],h,nbox,lmax,nmax,sortstates,sortenergies,wf)

    #Calculate energies as a test for levels
    occupied = 0
    for i in range(nmax):
        isospin = 0
        level = i
        i_s = sortstates[isospin,2,level]-1
        i_l = sortstates[isospin,1,level]
        i_n = sortstates[isospin,0,level]-1
        if(i_s < 0): continue
        occupied += 1
        #print("Level: {}, N: {}, L: {}, S: {}".format(level+1,i_n+1,i_l,i_s))
        #print("Energy: {}".format(np.sum(-h*wf[isospin,i_s,i_l,i_n,:]*hpsi[isospin,i_s,i_l,i_n,:])))

#print("There are {} occupied levels!".format(occupied))

In [126]:
global count
count = 0

start_time = time.time()
global prev_time 
prev_time = time.time()

train_num = len(train_wfs)

print("Training with {} WFs, each one with {} total states".format(train_num,sum(levels)))

ak = np.zeros((train_num,sum(levels)))
lam = np.zeros((sum(levels)))
coeffs = np.zeros((train_num*sum(levels) + sum(levels)))
coeffs[0:train_num*sum(levels)] = 0.5
coeffs[train_num*sum(levels):] = 25.0
#coeffs = sol.x

test_params = {"t0":-2471.45,"x0":0.77600,"t1":484.2300,"x1":-0.31700,"t2":-556.6900,"x2":-1.0,"t3":13654.0,"x3":1.2630,"w0":125.0,"sig":6.0}
#galerkin(coeffs,wfs,states,levels,test_params)

hybr_dict = {'col_deriv': 0, 'xtol': 1.49012e-8, 'maxfev': 500, 'band': None, 'eps': None, 'factor': 10, 'diag': None}
sol = optimize.root(galerkin, coeffs, args=(train_wfs,states,levels,default_params), method='hybr', options=hybr_dict)
print(sol.x)
print(galerkin(sol.x,train_wfs,states,levels,test_params))
print("Total time took: {}".format(time.time()-start_time))

Training with 5 WFs, each one with 13 total states
Iteration 1
Iteration time: 0.00032067298889160156
Iteration 2
Iteration time: 0.042723894119262695
Iteration 3
Iteration time: 0.041120290756225586
Iteration 4
Iteration time: 0.039496421813964844
Iteration 5
Iteration time: 0.03889155387878418
Iteration 6
Iteration time: 0.038802146911621094
Iteration 7
Iteration time: 0.03934931755065918
Iteration 8
Iteration time: 0.040062904357910156
Iteration 9
Iteration time: 0.03993654251098633
Iteration 10
Iteration time: 0.04005551338195801
Iteration 11
Iteration time: 0.03848767280578613
Iteration 12
Iteration time: 0.0377964973449707
Iteration 13
Iteration time: 0.039087533950805664
Iteration 14
Iteration time: 0.03939652442932129
Iteration 15
Iteration time: 0.04064464569091797
Iteration 16
Iteration time: 0.03894615173339844
Iteration 17
Iteration time: 0.03888368606567383
Iteration 18
Iteration time: 0.03841567039489746
Iteration 19
Iteration time: 0.04002547264099121
Iteration 20
Iterat

Iteration 167
Iteration time: 0.04006814956665039
Iteration 168
Iteration time: 0.04010176658630371
Iteration 169
Iteration time: 0.03871512413024902
[ 0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5
  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5
  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5
  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5
  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5 25.  25.  25.  25.  25.
 25.  25.  25.  25.  25.  25.  25.  25. ]
Iteration 170
Iteration time: 0.039513587951660156
[-4.50709560e+60 -1.21072564e+43 -1.32565844e+43 -3.84269339e+42
 -2.87806132e+43 -4.43510668e+42 -1.15915518e+42 -1.05087285e+60
 -1.27497746e+42 -1.38508885e+42 -4.54058168e+41 -2.64661895e+42
 -4.98322805e+41 -4.41079440e+60 -1.17801847e+43 -1.29055616e+43
 -3.72846339e+42 -2.87423017e+43 -4.32010396e+42 -1.12612929e+42
 -1.02842191e+60 -1.23910536e+42 -1.34697059e+42 -4.38440336e+41
 -2.63098433e

In [92]:
# Make the test WF with test_params
make_input(test_params,0)
with cwd('../HF/'):
    errors.append(run_HF("test",0))


In [124]:
# Make trial WF and hpsi with solved coeffs
trial_wf = make_wf(sol.x,train_wfs,states,levels)
trial_hpsi = get_hpsi(test_params,h,nbox,lmax,nmax,sorts,sorte,trial_wf)

# Read in the cheated test
restartfile = '../HF/results/wf_numpy.bin.test'
h,nbox,lmax,nmax,sorts,sorte,test_wf = read_wfs(restartfile)
test_hpsi = get_hpsi(test_params,h,nbox,lmax,nmax,sorts,sorte,test_wf)

for isospin in range(2):
        for j in range(levels[isospin]):
            i_s = sortstates[isospin,2,j]-1
            i_l = sortstates[isospin,1,j]
            i_n = sortstates[isospin,0,j]-1
            norm = np.dot(h*test_wf[isospin,i_s,i_l,i_n,:],trial_wf[isospin,i_s,i_l,i_n,:])
            trial_sp = np.dot(-h*trial_hpsi[isospin,i_s,i_l,i_n,:],trial_wf[isospin,i_s,i_l,i_n,:])
            test_sp = np.dot(-h*test_hpsi[isospin,i_s,i_l,i_n,:],test_wf[isospin,i_s,i_l,i_n,:])
            print("Trial: {trial} Test: {test} Difference: {diff} Norm: {norm}".format(trial=trial_sp,test=test_sp,diff=(trial_sp-test_sp)/trial_sp,norm=norm))



Trial: 1.6132257792438888e+78 Test: -46.51877073565693 Difference: 1.0 Norm: 2.3417842517013498e+17
Trial: 3.5577162558236616e+43 Test: -34.104299639512746 Difference: 1.0 Norm: 2.4374517715840547
Trial: 3.9497229409294756e+43 Test: -32.7605943857619 Difference: 1.0 Norm: 2.4384620128572427
Trial: 1.2274614800500944e+43 Test: -21.400228523237374 Difference: 1.0 Norm: 2.4041125141108903
Trial: 7.694178539084055e+43 Test: -17.786583792700007 Difference: 1.0 Norm: 2.331222262276933
Trial: 1.3996046319590166e+43 Test: -18.167316219392518 Difference: 1.0 Norm: 2.4276628796586515
Trial: 4.166314236648416e+42 Test: -9.270136025753626 Difference: 1.0 Norm: 2.3617479490587274
Trial: 9.739417402762726e+77 Test: -43.392751119623384 Difference: 1.0 Norm: 5.182681459775415e+17
Trial: 3.885492721073278e+42 Test: -32.77113764479229 Difference: 1.0 Norm: 2.408459406081137
Trial: 4.2491845017809653e+42 Test: -30.12199204142985 Difference: 1.0 Norm: 2.400492605105139
Trial: 1.4794017027721534e+42 Test: 